<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/PJ%232_Hybrid%20DL-ML%20(20230804).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
import numpy as np

In [56]:
# Load dataset
dataset = pd.read_csv("dataset.csv")
dataset.shape
dataset.isnull().sum()

Event             0
Time              0
S_Node            0
Node_id           0
Rest_Energy       0
Trace_Level       0
Mac_Type_Pckt     0
Source_IP_Port    0
Des_IP_Port       0
Packet_Size       0
TTL               0
Hop_Count         0
Broadcast_ID      0
Dest_Node_Num     0
Dest_Seq_Num      0
Src_Node_ID       0
Src_Seq_Num       0
Class             0
dtype: int64

In [58]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312106 entries, 0 to 312105
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Event           312106 non-null  int64  
 1   Time            312106 non-null  float64
 2   S_Node          312106 non-null  int64  
 3   Node_id         312106 non-null  int64  
 4   Rest_Energy     312106 non-null  float64
 5   Trace_Level     312106 non-null  int64  
 6   Mac_Type_Pckt   312106 non-null  int64  
 7   Source_IP_Port  312106 non-null  float64
 8   Des_IP_Port     312106 non-null  float64
 9   Packet_Size     312106 non-null  int64  
 10  TTL             312106 non-null  int64  
 11  Hop_Count       312106 non-null  int64  
 12  Broadcast_ID    312106 non-null  int64  
 13  Dest_Node_Num   312106 non-null  int64  
 14  Dest_Seq_Num    312106 non-null  int64  
 15  Src_Node_ID     312106 non-null  int64  
 16  Src_Seq_Num     312106 non-null  int64  
 17  Class     

In [59]:
dataset["Class"].unique()

array([0, 1, 2, 3])

In [60]:
#Installing imblearn
!pip install -U imbalanced-learn
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
# check version number
import imblearn
print(imblearn.__version__)

0.10.1


In [75]:
# Data preprocessing
y = dataset['Class'].values
X = dataset.drop(['Class'],axis=1)
X=X.values
print(X)
print(y)

[[1.00000000e+00 1.00000000e-01 7.90000000e+01 ... 0.00000000e+00
  7.90000000e+01 4.00000000e+00]
 [2.00000000e+00 1.00963167e-01 7.80000000e+01 ... 0.00000000e+00
  7.90000000e+01 4.00000000e+00]
 [2.00000000e+00 1.00963500e-01 7.60000000e+01 ... 0.00000000e+00
  7.90000000e+01 4.00000000e+00]
 ...
 [4.00000000e+00 6.57891729e+02 1.40000000e+02 ... 1.60000000e+01
  1.40000000e+02 7.60000000e+01]
 [4.00000000e+00 6.58574656e+02 1.40000000e+02 ... 1.60000000e+01
  1.40000000e+02 8.00000000e+01]
 [4.00000000e+00 6.79054956e+02 1.40000000e+02 ... 1.60000000e+01
  1.40000000e+02 9.00000000e+01]]
[0 0 0 ... 1 1 1]


In [63]:
import collections as c
counter = c.Counter(y)
print(counter)

Counter({0: 262851, 3: 29844, 1: 11766, 2: 7645})


In [64]:
y

array([0, 0, 0, ..., 1, 1, 1])

### **SMOTE**

In [65]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
counter = c.Counter(y)
print(counter)

Counter({0: 262851, 1: 262851, 2: 262851, 3: 262851})


### **Data Preprocessing for Train and Testing**

This code first loads the dataset and performs the necessary data preprocessing steps, including one-hot encoding the target variable and oversampling using SMOTE to address class imbalance.

Then, the dataset is split into training and testing sets, and the training set is reshaped for input to the CNN model.

The CNN model is built and compiled using the Keras library, and is then trained on the training set.

The CNN is then used to extract features from both the training and testing sets, which are then used as input to the RF model. The RF model is trained on the CNN features and evaluated on the testing set using classification report and confusion matrix.

Note that the final predictions in this code are made using the RF model on CNN features, not the CNN model itself.

In [66]:
from sklearn.model_selection import train_test_split    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [67]:
y_test

array([0, 0, 0, ..., 0, 0, 2])

In [71]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [73]:
# Data preprocessing
y = dataset['Class'].values
X = dataset.drop(['Class'], axis=1)
X = X.values

# One-hot encode y
y = to_categorical(y)

from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

# One-hot encode y
y = to_categorical(y)

# Reshape data for input to Conv2D
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1], 1))


model = Sequential()
model.add(Reshape((1, X_train.shape[2], 1), input_shape=(1, X_train.shape[2])))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
##history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Use CNN to extract features
X_train_cnn_features = cnn_model.predict(X_train_cnn)
X_test_cnn_features = cnn_model.predict(X_test_cnn)


Epoch 1/10
26286/26286 [==============================] - 160s 6ms/step - loss: 0.2900 - accuracy: 0.8838 - val_loss: 0.2275 - val_accuracy: 0.9169
Epoch 2/10
26286/26286 [==============================] - 180s 7ms/step - loss: 0.1907 - accuracy: 0.9276 - val_loss: 0.1417 - val_accuracy: 0.9430
Epoch 3/10
26286/26286 [==============================] - 207s 8ms/step - loss: 0.1771 - accuracy: 0.9337 - val_loss: 0.1539 - val_accuracy: 0.9387
Epoch 4/10
26286/26286 [==============================] - 206s 8ms/step - loss: 0.1694 - accuracy: 0.9369 - val_loss: 0.1229 - val_accuracy: 0.9539
Epoch 5/10
26286/26286 [==============================] - 205s 8ms/step - loss: 0.1524 - accuracy: 0.9441 - val_loss: 0.3729 - val_accuracy: 0.9027
Epoch 6/10
26286/26286 [==============================] - 213s 8ms/step - loss: 0.1399 - accuracy: 0.9508 - val_loss: 0.0951 - val_accuracy: 0.9688
Epoch 7/10
26286/26286 [==============================] - 210s 8ms/step - loss: 0.1337 - accuracy: 0.9535 - val_

In [76]:
# Use CNN to extract features
X_train_cnn_features = cnn_model.predict(X_train_cnn)
X_test_cnn_features = cnn_model.predict(X_test_cnn)

# Flatten the last 2 dimensions of X_train_cnn_features
X_train_cnn_features = np.reshape(X_train_cnn_features, (X_train_cnn_features.shape[0], -1))
X_test_cnn_features = np.reshape(X_test_cnn_features, (X_test_cnn_features.shape[0], -1))

# Build RF model
rf_model = RandomForestClassifier(n_estimators=100, random_state=101)

# Train RF model on CNN features
rf_model.fit(X_train_cnn_features, np.argmax(y_train, axis=1))


6572/6572 [==============================] - 8s 1ms/step


RandomForestClassifier(random_state=101)

In [77]:
# Build RF model
##rf_model = RandomForestClassifier(n_estimators=100, random_state=101)

# Train RF model on CNN features
##rf_model.fit(X_train_cnn_features, np.argmax(y_train, axis=1))

# Evaluate RF model on CNN features
y_pred_rf = rf_model.predict(X_test_cnn_features)
y_test_rf = np.argmax(y_test, axis=1)
print(classification_report(y_test_rf, y_pred_rf))
print(confusion_matrix(y_test_rf, y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     52500
           1       1.00      1.00      1.00     52554
           2       1.00      1.00      1.00     52545
           3       1.00      1.00      1.00     52682

    accuracy                           1.00    210281
   macro avg       1.00      1.00      1.00    210281
weighted avg       1.00      1.00      1.00    210281

[[52495     1     4     0]
 [   24 52522     0     8]
 [    1     3 52536     5]
 [    0    19     5 52658]]
